<a href="https://colab.research.google.com/github/machinehistories/3d_topography_generator_notebook/blob/main/mesh_from_place.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================
# 🌍 SRTM to 3D Mesh Generator (Colab, fixed)
# Avoids make errors by downloading tiles directly
# ==========================================

!pip install geopy rasterio numpy trimesh pyproj shapely elevation requests tqdm

from google.colab import drive
drive.mount('/content/drive')

from google.colab import output
output.enable_custom_widget_manager()

import os
import numpy as np
import rasterio
from rasterio.mask import mask
from geopy.geocoders import Nominatim
from shapely.geometry import box, mapping
from pyproj import Geod, Transformer
import trimesh
import requests
from tqdm import tqdm
import zipfile

# --- Step 1: User Inputs ---
PLACE_NAME = "Mount Shasta, California"
SIZE_METERS = 10000
OUTPUT_FORMAT = "obj"
SAVE_DIR = "/content/drive/MyDrive/SRTM_Meshes"
os.makedirs(SAVE_DIR, exist_ok=True)

# --- Step 2: Geocode the place name ---
geolocator = Nominatim(user_agent="srtm_to_mesh_colab")
location = geolocator.geocode(PLACE_NAME)
if not location:
    raise ValueError(f"Could not find location '{PLACE_NAME}'")
lat, lon = location.latitude, location.longitude
print(f"📍 Found location: {PLACE_NAME} at ({lat:.5f}, {lon:.5f})")

# --- Step 3: Compute bounding box in degrees ---
geod = Geod(ellps="WGS84")
lon_min, _, _ = geod.fwd(lon, lat, -90, SIZE_METERS / 2)
lon_max, _, _ = geod.fwd(lon, lat, 90, SIZE_METERS / 2)
_, lat_min, _ = geod.fwd(lon, lat, 180, SIZE_METERS / 2)
_, lat_max, _ = geod.fwd(lon, lat, 0, SIZE_METERS / 2)
bounds = (min(lon_min, lon_max), min(lat_min, lat_max), max(lon_min, lon_max), max(lat_min, lat_max))
print(f"🧭 Bounding Box: {bounds}")

# --- Step 4: Download SRTM data directly from NASA (AWS) ---
# Use 1 arc-second data (~30m). We’ll compute the tile name automatically.
tile_lat = int(np.floor(lat))
tile_lon = int(np.floor(lon))
ns = "N" if tile_lat >= 0 else "S"
ew = "E" if tile_lon >= 0 else "W"
tile_name = f"{ns}{abs(tile_lat):02d}{ew}{abs(tile_lon):03d}"
url = f"https://s3.amazonaws.com/elevation-tiles-prod/skadi/{ns}{abs(tile_lat):02d}/{tile_name}.hgt.gz"

dem_gz = f"/content/{tile_name}.hgt.gz"
dem_hgt = dem_gz.replace(".gz", "")

# Download if not already cached
if not os.path.exists(dem_hgt):
    print(f"⬇️ Downloading {tile_name} from {url}")
    r = requests.get(url, stream=True)
    if r.status_code != 200:
        raise ValueError("Could not download SRTM tile (tile may not exist for this region).")
    with open(dem_gz, "wb") as f:
        for chunk in tqdm(r.iter_content(chunk_size=8192)):
            f.write(chunk)
    import gzip, shutil
    with gzip.open(dem_gz, "rb") as f_in, open(dem_hgt, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)
    print("✅ Decompressed DEM")

# --- Step 5: Read and crop the DEM ---
with rasterio.open(dem_hgt, driver="SRTMHGT") as src:
    bbox_geom = box(*bounds)
    out_image, out_transform = mask(src, [mapping(bbox_geom)], crop=True)
    data = out_image[0]
    transform = out_transform

print(f"✅ DEM cropped: shape={data.shape}")

# --- Step 6: Convert DEM to mesh safely ---

ny, nx = data.shape
data = np.where(data < -1000, np.nan, data)  # mark invalid data

xs = np.arange(nx)
ys = np.arange(ny)
xv, yv = np.meshgrid(xs, ys)
lon_grid, lat_grid = rasterio.transform.xy(transform, yv, xv)
lon_grid = np.array(lon_grid)
lat_grid = np.array(lat_grid)

# Convert lat/lon to meters
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
x_m, y_m = transformer.transform(lon_grid, lat_grid)

# Flatten arrays
X = x_m.flatten()
Y = y_m.flatten()
Z = data.flatten()

# Create vertices (keeping full grid)
vertices = np.column_stack((X, Y, np.nan_to_num(Z, nan=0)))

# Build faces safely (skip NaNs)
faces = []
cols = nx
rows = ny
for j in range(rows - 1):
    for i in range(cols - 1):
        idx = j * cols + i
        # Indices of the 4 corners of the cell
        v1, v2, v3, v4 = idx, idx + 1, idx + cols, idx + cols + 1
        # Skip if any vertex has NaN elevation
        if np.isnan(Z[v1]) or np.isnan(Z[v2]) or np.isnan(Z[v3]) or np.isnan(Z[v4]):
            continue
        faces.append([v1, v2, v3])
        faces.append([v2, v4, v3])

faces = np.array(faces, dtype=np.int32)

mesh = trimesh.Trimesh(vertices=vertices, faces=faces, process=False)
print(f"✅ Mesh generated with {len(vertices)} vertices and {len(faces)} faces")

# --- Step 7: Export Mesh ---
filename_base = PLACE_NAME.replace(",", "").replace(" ", "_")
output_path = os.path.join(SAVE_DIR, f"{filename_base}.{OUTPUT_FORMAT}")
mesh.export(output_path)
print(f"💾 Saved mesh to: {output_path}")

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()